In this notebook we calculate Stored Carbon for the ISCN available here (registration/login necessary):

https://iscn.fluxdata.org/data/access-data/database-reports/

I've used the 'ISCN_SOC-DATA_LAYER_1-1' dataset for everything but Bulk Density values, that were instead taken from 'ISCNTemplate_NRCS_BD_predictions'. 

Stored organic carbon has been calculated only for non-organic horizons, so the value 'be_pred_2' has been used.

I also check for layers continuity (still WIP).

In [23]:
using DataFrames, CSV, Query, StatsBase

Importing layer samplings data:

In [24]:
df_ISCN_orig = CSV.read("ISCN_SOC-DATA_LAYER_1-1.csv", DataFrame)

┌ Warning: thread = 1 warning: only found 93 / 94 columns around data row: 227765. Filling remaining columns with `missing`
└ @ CSV /Users/Daniele/.julia/packages/CSV/la2cd/src/file.jl:603


,dataset_name_sub,dataset_name_soc,lat (dec. deg)
,String,String,String?
1,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219"
2,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219"
3,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219"
4,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219"
5,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219"
6,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219"
7,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219"
8,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219"
9,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219"


In [25]:
count(ismissing,(df_ISCN_orig[:,:hzn_desgn_other])) #227764 - all missing values, dropping this column
count(ismissing,(df_ISCN_orig[:,:hzn_desgn]))  		#13812  - ok
count(ismissing,(df_ISCN_orig[:,:hzn])) 			#12244  - ok

12244

Cleaning up layers data:

In [26]:
#copy of original dataset and tidied up column names
df_ISCN = deepcopy(df_ISCN_orig)
rename!(df_ISCN,names(df_ISCN) .=> tidy_names(names(df_ISCN)))

df_ISCN = df_ISCN |>

#pick relevant columns
@select(:site_name,:profile_name,:layer_name,:lat_dec_deg,:long_dec_deg,:datum_datum,
        :observation_date_yyyymmdd,:hzn,:hzn_desgn,:layer_top_cm,:layer_bot_cm,:oc_percent) |>

#remove layers with missing info
@dropna(:layer_top_cm,:layer_bot_cm,:oc_percent) |>

#keep layers for which top < bot
@filter(_.layer_top_cm < _.layer_bot_cm) |>

#keep only non-organic layers
#replacing missings with Z necessary for comparison
#organic layers contain 'O' or 'o' in the horizon designation
@replacena(:hzn=>"Z") |>
@filter(!occursin("O",_.hzn) && !occursin("o",_.hzn)) |>

#repeat the same above for the other designation column
@replacena(:hzn_desgn=>"Z") |>
@filter(!occursin("O",_.hzn_desgn) && !occursin("o",_.hzn_desgn)) |>

#parse oc_percent values and keep only non-negative ones
@mutate(oc_percent = replace(_.oc_percent,','=>'.')) |>
@mutate(oc_percent = parse(Float64, _.oc_percent)) |>
@filter(_.oc_percent >= 0) |>

#parse latitude values
#missings are replaced with 0 to allow parsing, but we keep layers with missing lat/long
@replacena(:lat_dec_deg=>"0") |>
@mutate(lat_dec_deg = replace(_.lat_dec_deg,','=>'.')) |>
@mutate(lat_dec_deg = parse(Float64, _.lat_dec_deg)) |>

#parse longitude values 
@replacena(:long_dec_deg=>"0") |>
@mutate(long_dec_deg = replace(_.long_dec_deg,','=>'.')) |>
@mutate(long_dec_deg = parse(Float64, _.long_dec_deg)) |>

#keep only unique sites/profiles/layers
@unique((_.site_name, _.profile_name, _.layer_name)) |>

DataFrame

,site_name,profile_name,layer_name,lat_dec_deg,long_dec_deg,datum_datum,observation_date_yyyymmdd
,String,String,String,Float64,Float64,String?,String?
1,50ND075005,40A0001,40A00001,48.805,-101.735,WGS84,19/09/1950
2,50ND075005,40A0001,40A00002,48.805,-101.735,WGS84,19/09/1950
3,50ND075005,40A0001,40A00003,48.805,-101.735,WGS84,19/09/1950
4,50ND075005,40A0001,40A00004,48.805,-101.735,WGS84,19/09/1950
5,50ND075005,40A0001,40A00005,48.805,-101.735,WGS84,19/09/1950
6,50ND075005,40A0001,40A00006,48.805,-101.735,WGS84,19/09/1950
7,50ND075005,40A0001,40A00007,48.805,-101.735,WGS84,19/09/1950
8,50ND075003,40A0013,40A00094,48.9366,-101.585,WGS84,18/09/1950
9,50ND075003,40A0013,40A00095,48.9366,-101.585,WGS84,18/09/1950


In [27]:
function tidy_names(old_names)

return new_names = old_names |>
    #remove leading and trailing spaces
    n -> strip.(n) |>

    #replace spaces with underscores
    n -> replace.(n, ' ' => '_') |> 

    #remove parenthesis
    n -> replace.(n,'(' => "") |> 
    n -> replace.(n,')' => "") |>

    #remove dashes and dots
    n -> replace.(n,'-' => "") |> 
    n -> replace.(n, '.' => "") |>

    #all lowercase
    n -> lowercase.(n)

end

tidy_names (generic function with 1 method)

In [28]:
function continuity_check(profile) 
    #the function assumes layers are ordered (e.g.  0 -> 20 -> 40)

    top = profile.layer_top_cm
    bot = profile.layer_bot_cm

    n_layers = length(top)

    #1st layer is always good, from 2nd we check for matching depths at bot/top
    mask = append!([true],[top[l] == bot[l-1] for l in 2:n_layers])

    #if we run into a non-matching layer, we mark as 'false' all following ones
    first_noMatch_id = findfirst(!, mask)

    if isnothing(first_noMatch_id) == false
        mask[first_noMatch_id:end] .= false
    end

    return mask
end

continuity_check (generic function with 1 method)

In [29]:
sp_key_ISCN = unique([(row.site_name, row.profile_name) for row in eachrow(df_ISCN)])

18387-element Array{Tuple{String,String},1}:
 ("50ND075005", "40A0001")
 ("50ND075003", "40A0013")
 ("50ND075007", "40A0067")
 ("59ND045001", "40A0102")
 ("50ND075006", "40A0107")
 ("50ND075008", "40A0111")
 ("50ND075009", "40A0130")
 ("50ND075010", "40A0131")
 ("S1949CO049005", "40A0138")
 ("S1949CO049007", "40A0140")
 ("S1964TN157003", "40A0158")
 ("S1958WV025003", "40A0162")
 ("67FN220000", "40A0165")
 ⋮
 ("uiuc198801942", "uiuc198801942")
 ("uiuc198801967", "uiuc198801967")
 ("uiuc198801968", "uiuc198801968")
 ("uiuc198801969", "uiuc198801969")
 ("uiuc198801970", "uiuc198801970")
 ("uiuc198801972", "uiuc198801972")
 ("uiuc198801973", "uiuc198801973")
 ("uiuc198902031", "uiuc198902031")
 ("uiuc198902034", "uiuc198902034")
 ("uiuc198902035", "uiuc198902035")
 ("uiuc198902036", "uiuc198902036")
 ("uiuc199002049", "uiuc199002049")

In [30]:
#generate continuity layers mask	
layers_keep = Vector{Bool}(undef,0)

for (s,p) in sp_key_ISCN

    append!(layers_keep,continuity_check(df_ISCN[(df_ISCN.site_name.==s) .& (df_ISCN.profile_name.==p),:]))

end

In [31]:
#apply mask to define new DF
df_ISCN_mask = df_ISCN[layers_keep,:]

,site_name,profile_name,layer_name,lat_dec_deg,long_dec_deg,datum_datum,observation_date_yyyymmdd
,String,String,String,Float64,Float64,String?,String?
1,50ND075005,40A0001,40A00001,48.805,-101.735,WGS84,19/09/1950
2,50ND075005,40A0001,40A00002,48.805,-101.735,WGS84,19/09/1950
3,50ND075005,40A0001,40A00003,48.805,-101.735,WGS84,19/09/1950
4,50ND075005,40A0001,40A00004,48.805,-101.735,WGS84,19/09/1950
5,50ND075005,40A0001,40A00005,48.805,-101.735,WGS84,19/09/1950
6,50ND075005,40A0001,40A00006,48.805,-101.735,WGS84,19/09/1950
7,50ND075005,40A0001,40A00007,48.805,-101.735,WGS84,19/09/1950
8,50ND075003,40A0013,40A00094,48.9366,-101.585,WGS84,18/09/1950
9,50ND075003,40A0013,40A00095,48.9366,-101.585,WGS84,18/09/1950


Importing set with BD values:

In [32]:
df_BD_pred_orig = CSV.read("ISCNTemplate_NRCS_BD_predictions.csv", DataFrame)

┌ Warning: thread = 1 warning: only found 26 / 27 columns around data row: 434583. Filling remaining columns with `missing`
└ @ CSV /Users/Daniele/.julia/packages/CSV/la2cd/src/file.jl:603


,dataset_name,site_name,profile_name,layer_name,layer_top,layer_bot,hzn_desgn
,String,String,String,String,String,String,String?
1,Boby_Mack,BF-61,BF-61-1,BF61-1-F,0,4,unknown
2,Boby_Mack,BF-61,BF-61-1,BF61-1-H,4,6,unknown
3,Boby_Mack,BF-61,BF-61-1,BF61-1-M1,6,11,unknown
4,Boby_Mack,BF-61,BF-61-1,BF61-1-M2,11,16,unknown
5,Boby_Mack,BF-61,BF-61-2,BF61-2-DM,0,5,o
6,Boby_Mack,BF-61,BF-61-2,BF61-2-F,5,19,unknown
7,Boby_Mack,BF-61,BF-61-2,BF61-2-H,19,22,unknown
8,Boby_Mack,BF-61,BF-61-2,BF61-2-H:354,22,25,unknown
9,Boby_Mack,BF-61,BF-61-2,BF61-2-M1,25,30,unknown


Cleaning up BD set:

In [33]:
#copy of original dataset and tidied up column names
df_BD_pred = deepcopy(df_BD_pred_orig)

df_BD_pred = df_BD_pred |>

#pick relevant columns
@select(:site_name,:profile_name,:layer_name,:bd_pred_2) |>

#remove layers with missing predicted BD
@dropna(:bd_pred_2) |>

#parse bd_pred_2 values and keep only non-negative ones
@mutate(bd_pred_2 = replace(_.bd_pred_2,','=>'.')) |>
@mutate(bd_pred_2 = parse(Float64, _.bd_pred_2)) |>
@filter(_.bd_pred_2 > 0) |>

DataFrame

,site_name,profile_name,layer_name,bd_pred_2
,String,String,String,Float64
1,BF-61,BF-61-1,BF61-1-F,0.73
2,BF-61,BF-61-1,BF61-1-H,0.74
3,BF-61,BF-61-1,BF61-1-M1,1.03
4,BF-61,BF-61-1,BF61-1-M2,1.13
5,BF-61,BF-61-2,BF61-2-DM,0.36
6,BF-61,BF-61-2,BF61-2-F,0.67
7,BF-61,BF-61-2,BF61-2-H,0.77
8,BF-61,BF-61-2,BF61-2-H:354,0.67
9,BF-61,BF-61-2,BF61-2-M1,0.96


NOTE: ‘bd_pred_1’ is for organic horizons (hzn = O), ‘be_pred_2’ for mineral soil horizons (hzn != O).


In [34]:
#adding predicted BD values to each layer
df_merged = innerjoin(df_ISCN_mask, df_BD_pred, on = [:site_name, :profile_name, :layer_name])

#making sure there are no missing BD values and no duplicated layers
dropmissing!(df_merged,:bd_pred_2)
unique!(df_merged, [:site_name, :profile_name, :layer_name])

,site_name,profile_name,layer_name,lat_dec_deg,long_dec_deg,datum_datum,observation_date_yyyymmdd
,String,String,String,Float64,Float64,String?,String?
1,50ND075005,40A0001,40A00001,48.805,-101.735,WGS84,19/09/1950
2,50ND075005,40A0001,40A00002,48.805,-101.735,WGS84,19/09/1950
3,50ND075005,40A0001,40A00003,48.805,-101.735,WGS84,19/09/1950
4,50ND075005,40A0001,40A00004,48.805,-101.735,WGS84,19/09/1950
5,50ND075005,40A0001,40A00005,48.805,-101.735,WGS84,19/09/1950
6,50ND075005,40A0001,40A00006,48.805,-101.735,WGS84,19/09/1950
7,50ND075005,40A0001,40A00007,48.805,-101.735,WGS84,19/09/1950
8,50ND075003,40A0013,40A00094,48.9366,-101.585,WGS84,18/09/1950
9,50ND075003,40A0013,40A00095,48.9366,-101.585,WGS84,18/09/1950


In [35]:
#keys to wrangle profiles/layers
sp_key_merged = unique([(row.site_name, row.profile_name) for row in eachrow(df_merged)])

18386-element Array{Tuple{String,String},1}:
 ("50ND075005", "40A0001")
 ("50ND075003", "40A0013")
 ("50ND075007", "40A0067")
 ("59ND045001", "40A0102")
 ("50ND075006", "40A0107")
 ("50ND075008", "40A0111")
 ("50ND075009", "40A0130")
 ("50ND075010", "40A0131")
 ("S1949CO049005", "40A0138")
 ("S1949CO049007", "40A0140")
 ("S1964TN157003", "40A0158")
 ("S1958WV025003", "40A0162")
 ("67FN220000", "40A0165")
 ⋮
 ("uiuc198801942", "uiuc198801942")
 ("uiuc198801967", "uiuc198801967")
 ("uiuc198801968", "uiuc198801968")
 ("uiuc198801969", "uiuc198801969")
 ("uiuc198801970", "uiuc198801970")
 ("uiuc198801972", "uiuc198801972")
 ("uiuc198801973", "uiuc198801973")
 ("uiuc198902031", "uiuc198902031")
 ("uiuc198902034", "uiuc198902034")
 ("uiuc198902035", "uiuc198902035")
 ("uiuc198902036", "uiuc198902036")
 ("uiuc199002049", "uiuc199002049")

In [36]:
function orgc_storage(profile) 

    top = profile.layer_top_cm
    bot = profile.layer_bot_cm

    blk = profile.bd_pred_2 #[g/cm³]
    orgc = profile.oc_percent #[%]

    layers_heights = (bot.-top)

    mass=0.0
    for layer in 1:length(layers_heights)
        #calculation for 100cmx100cm of surface, for all layers: volume x carbon/volume
        mass += (100*100*layers_heights[layer])*(blk[layer]*orgc[layer]/100)/1000 #result in [Kg]
    end

    return mass
end

orgc_storage (generic function with 1 method)

In [37]:
#computing stored ORGC

computed_orgc = Dict()

for (s,p) in sp_key_merged
    profile_calc = df_merged[(df_merged.site_name .== s) .& (df_merged.profile_name .== p), :]
    computed_orgc[(s,p)] = orgc_storage(profile_calc)
end

#storing results in DF	
df_orgc = DataFrame(site_name = [s for (s,_) in sp_key_merged], 
                    profile_name = [p for (_,p) in sp_key_merged], 
                    orgc_kg = Float64.(values(computed_orgc)))


,site_name,profile_name,orgc_kg
,String,String,Float64
1,50ND075005,40A0001,3.81415
2,50ND075003,40A0013,8.54735
3,50ND075007,40A0067,3.94668
4,59ND045001,40A0102,18.5023
5,50ND075006,40A0107,6.55938
6,50ND075008,40A0111,9.44642
7,50ND075009,40A0130,5.94671
8,50ND075010,40A0131,10.0302
9,S1949CO049005,40A0138,12.5975


In [38]:
#retrieving min/max depth per profile 

depth(profile) = (minimum(profile.layer_top_cm), maximum(profile.layer_bot_cm))


profiles_depth = Dict()

for (s,p) in sp_key_merged
profiles_depth[(s,p)] = depth(df_merged[(df_merged.site_name .== s) .& (df_merged.profile_name .== p), :])
end

df_depths = sort!(DataFrame(site_name = [s for (s,_) in sp_key_merged],
                            profile_name = profile_name = [p for (_,p) in sp_key_merged],
                            min_depth = [min for (min,_) in values(profiles_depth)],
                            max_depth = [max for (_,max) in values(profiles_depth)]))




,site_name,profile_name,min_depth,max_depth
,String,String,Int64,Int64
1,01NE031001,40A3476,33,64
2,01NE079001-OSD,94P0105,12,119
3,02KY195004,83P0647,0,152
4,03MT021001,91P0898,0,64
5,04-PMP-03,90P1050,0,100
6,04ut626-122lab,92P0737,10,152
7,04ut626-123lab,92P0738,0,5
8,07-CBR-002,40A1051,0,101
9,08CO073026,90P0037,0,120


In [39]:
#covariates from starting dataset
df_lat_long_date = unique(df_merged[:,[:site_name,:profile_name,:lat_dec_deg,:long_dec_deg,:observation_date_yyyymmdd]]);

In [40]:
#first join
df_join_A = innerjoin(df_orgc,df_lat_long_date, on = [:site_name,:profile_name])

,site_name,profile_name,orgc_kg,lat_dec_deg,long_dec_deg,observation_date_yyyymmdd
,String,String,Float64,Float64,Float64,String?
1,50ND075005,40A0001,3.81415,48.805,-101.735,19/09/1950
2,50ND075003,40A0013,8.54735,48.9366,-101.585,18/09/1950
3,50ND075007,40A0067,3.94668,48.8083,-101.707,19/09/1950
4,59ND045001,40A0102,18.5023,46.4494,-98.37,21/08/1959
5,50ND075006,40A0107,6.55938,48.8211,-101.747,19/09/1950
6,50ND075008,40A0111,9.44642,48.8991,-101.554,20/09/1950
7,50ND075009,40A0130,5.94671,48.9427,-101.587,20/09/1950
8,50ND075010,40A0131,10.0302,48.9153,-101.606,20/09/1950
9,S1949CO049005,40A0138,12.5975,0.0,0.0,02/09/1949


In [41]:
df_final = innerjoin(df_join_A, df_depths, on = [:site_name,:profile_name])

,site_name,profile_name,orgc_kg,lat_dec_deg,long_dec_deg,observation_date_yyyymmdd
,String,String,Float64,Float64,Float64,String?
1,50ND075005,40A0001,3.81415,48.805,-101.735,19/09/1950
2,50ND075003,40A0013,8.54735,48.9366,-101.585,18/09/1950
3,50ND075007,40A0067,3.94668,48.8083,-101.707,19/09/1950
4,59ND045001,40A0102,18.5023,46.4494,-98.37,21/08/1959
5,50ND075006,40A0107,6.55938,48.8211,-101.747,19/09/1950
6,50ND075008,40A0111,9.44642,48.8991,-101.554,20/09/1950
7,50ND075009,40A0130,5.94671,48.9427,-101.587,20/09/1950
8,50ND075010,40A0131,10.0302,48.9153,-101.606,20/09/1950
9,S1949CO049005,40A0138,12.5975,0.0,0.0,02/09/1949


In [42]:
CSV.write("stored_orgc_ISCN.csv", df_final)

"stored_orgc_ISCN.csv"

Results here: https://drive.google.com/file/d/1z2rm7TpxoBf8Lpk9artj6-2bzfIkK1_T/view?usp=sharing